In [1]:
#!pip -q install newspaper
#!pip -q install selenium
#!pip -q install openai

In [2]:
import requests
from newspaper import Article
from datetime import datetime, timedelta
import os
import json

from openai import OpenAI

import numpy as np

from modules.newsapi import NewsCollector
from modules.agent import FinRock

In [3]:
start_date = "2025-11-04"
end_date = "2025-11-04"

In [4]:
current_news = NewsCollector(start_date, end_date)
current_news.collect()

0 portal ---> https://www.bbc.com/news/articles/cwyk6kvyxvzo
content ---> <html lang="en-GB"><
1 portal ---> https://www.rockstargames.com/newswire/article/ak3ak31a49a221/grand-theft-auto-vi-is-now-set-to-launch-november-19-2026
content ---> <html lang="en_us"><
2 portal ---> https://www.nbcnews.com/news/obituaries/dallas-cowboys-marshawn-kneeland-dies-rcna242340
content ---> <html lang="en"><hea
3 portal ---> https://www.politico.com/live-updates/2025/11/06/congress/shutdown-vote-senate-friday-00640360
content ---> <html lang="en-US" d
4 portal ---> https://www.cnbc.com/2025/11/06/obesity-drug-prices-trump-deals-eli-lilly-novo-nordisk.html
content ---> <html lang="en" pref
5 portal ---> https://www.nbcnews.com/politics/justice-department/dc-sandwich-guy-verdict-rcna242142
content ---> <html lang="en"><hea
6 portal ---> https://www.washingtonpost.com/politics/2025/11/06/nancy-pelosi-retire-2025-house-speaker-democrats/
content ---> <html dir="ltr" lang
7 portal ---> https://www.npr.org

In [5]:
current_news.data

{'status': 'ok',
 'totalResults': 35,
 'articles': [{'source': {'id': None, 'name': 'BBC News'},
   'author': None,
   'title': "Elon Musk's $1tn pay deal approved by Tesla shareholders - BBC",
   'description': 'The richest man in the world will get hundreds of millions of new shares if he hits his targets.',
   'url': 'https://www.bbc.com/news/articles/cwyk6kvyxvzo',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/fa6b/live/01ac1f30-bb5f-11f0-9872-515b67a0dae0.jpg',
   'publishedAt': '2025-11-06T21:55:02Z',
   'content': 'Lily JamaliNorth America Technology correspondent\r\nTesla shareholders have approved a record-breaking pay package for boss Elon Musk that could be worth nearly $1tn (£760bn).\r\nThe unprecedented deal … [+2934 chars]',
   'full_content': 'Elon Musk\'s $1tn pay deal approved by Tesla shareholders - BBC\nLily JamaliNorth America Technology correspondent\r\nTesla shareholders have approved a record-breaking pay package for boss Elon Musk that could 

In [6]:
fr = FinRock(current_news.data)

In [7]:
news = """
El BBVA fracasa en su opa sobre el Sabadell al lograr solo un 25% de aceptación del capital
El banco vasco no alcanza el nivel mínimo del 30% que había fijado en la oferta para seguir adelante con la operación

Carlos Torres (BBVA): "No se ha alcanzado el nivel mínimo de aceptación fijado"
El CEO del Banco Sabadell, César González-Bueno (izquierda), y el presidente del Banco Sabadell, Josep Oliu (derecha).

La guerra entre el BBVA y el Sabadell se salda con victoria del pequeño. El banco vasco ha conseguido una aceptación en 
la oferta del 25,33% de las acciones y del 25,47% de los derechos de voto, porcentaje que incluye la autocartera, según 
ha confirmado la Comisión Nacional del Mercado de Valores (CNMV) en la noche de este jueves. Niveles en todo caso inferiores 
al 30%, el mínimo que necesitaba para que pudiera seguir adelante en su asalto al catalán.
"""

In [23]:
news = """
BBVA México lanzó una nueva aplicación móvil que presenta mejoras significativas en velocidad, seguridad e intuición, gracias a la inteligencia artificial. Entre sus nuevas funciones se incluye la posibilidad de personalizar la pantalla de inicio, acceso rápido a funciones como pagos y transferencias, y la opción de ocultar datos sensibles en público.  
"""

In [ ]:
stocks2sectors = fr.get_sectors()

sectors2stocks = {}
for k, v in stocks2sectors.items():
    sectors2stocks[v] = sectors2stocks.get(v, []) + [k]
#news = "China amenaza con invadir taiwan, el principal proveedor de microchips."
out = fr.analize_news(news, sectors2stocks)

In [ ]:
out

In [12]:
#url = "https://elpais.com/economia/2025-10-16/el-sabadell-vence-al-bbva-y-consigue-el-apoyo-de-los-inversores-para-seguir-en-solitario.html"

In [13]:
results, affected = fr.get_insights()

{'news_headline': 'Tesla shareholders approve Elon Musk’s record pay package; renewed focus on AI, robotaxi and Optimus', 'affected_sectors': [{'sector': {'Automotive': ['Tesla']}, 'stock_price_effect': 'Mild Up', 'reasoning': 'Approval removes CEO-retention and governance overhang, boosting sentiment and AI/robotaxi optionality for Tesla despite ongoing EV-demand and FSD regulatory risks.'}, {'sector': {'GPU Providers': ['NVIDIA']}, 'stock_price_effect': 'Mild Up', 'reasoning': 'Tesla’s emphasis on FSD/robotaxi/Optimus implies continued heavy AI compute needs, a modest sentiment tailwind for NVIDIA given its role in supplying GPUs.'}], 'unaffected_sectors': ['Semiconductors', 'Banking', 'Neobank', 'Fintech', 'Agriculture', 'Food Production', 'Energy', 'Energy Services', 'Tech', 'E-commerce', 'Enterprise Software', 'Electric Vehicles', 'Pharma', 'Retail', 'Luxury Goods', 'Apparel', 'Telecom', 'Aerospace', 'Defense', 'Crypto', 'Foreign Exchange'], 'url': 'https://www.bbc.com/news/articl

In [ ]:
for r in results:
    if r['affected_sectors']:
        for i in range(len(r['affected_sectors'])):
            if r['affected_sectors'][i]['stock_price_effect'] == 'Strong Down':
                print("Strong Down effect:", r['affected_sectors'][i]['sector'])
                print("news_headline:", r['news_headline'])
                print("reasoning:", r['affected_sectors'][i]['reasoning'])
                print("="*80)

In [21]:
for r in results:
    if r['affected_sectors']:
        for i in range(len(r['affected_sectors'])):
            if r['affected_sectors'][i]['stock_price_effect'] == 'Mild Down':
                print("Mild Down effect:", r['affected_sectors'][i]['sector'])
                print("Mild Down effect:", r['news_headline'])
                print("Mild Down effect:", r['affected_sectors'][i]['reasoning'])
                print("="*80)

Mild Down effect: {'Food Production': ['Nestlé', 'Unilever', 'Tyson Foods']}
Mild Down effect: Trump strikes pricing deals with Eli Lilly and Novo Nordisk for GLP-1 weight-loss drugs; Medicare to cover obesity use in 2026; DTC pricing cut
Mild Down effect: Cheaper and broader access to GLP-1s likely accelerates adoption, pressuring caloric intake and snacking volumes. Near-term sentiment risk for packaged food/protein producers outweighs any offset.
Mild Down effect: {'Retail': ['Walmart', 'Costco', 'Target']}
Mild Down effect: Trump strikes pricing deals with Eli Lilly and Novo Nordisk for GLP-1 weight-loss drugs; Medicare to cover obesity use in 2026; DTC pricing cut
Mild Down effect: GLP-1 adoption can reduce grocery basket sizes and discretionary food purchases; pharmacy script upside is partially offset by DTC channels (e.g., TrumpRx), yielding modest net pressure for grocery-heavy retailers.
Mild Down effect: {'Fintech': ['Adyen', 'Stripe', 'PayPal', 'Wise']}
Mild Down effect: FA

In [16]:
for r in results:
    if r['affected_sectors']:
        for i in range(len(r['affected_sectors'])):
            if r['affected_sectors'][i]['stock_price_effect'] == 'Strong Up':
                print("Strong Up effect:", r['affected_sectors'][i]['sector'])
                print("Strong Up effect:", r['news_headline'])
                print("Strong Up effect:", r['affected_sectors'][i]['reasoning'])
                print("="*80)

In [22]:
for r in results:
    if r['affected_sectors']:
        for i in range(len(r['affected_sectors'])):
            if r['affected_sectors'][i]['stock_price_effect'] == 'Mild Up':
                print("Mild Up effect:", r['affected_sectors'][i]['sector'])
                print("Mild Up effect:", r['news_headline'])
                print("Mild Up effect:", r['affected_sectors'][i]['reasoning'])
                print("="*80)

Mild Up effect: {'Automotive': ['Tesla']}
Mild Up effect: Tesla shareholders approve Elon Musk’s record pay package; renewed focus on AI, robotaxi and Optimus
Mild Up effect: Approval removes CEO-retention and governance overhang, boosting sentiment and AI/robotaxi optionality for Tesla despite ongoing EV-demand and FSD regulatory risks.
Mild Up effect: {'GPU Providers': ['NVIDIA']}
Mild Up effect: Tesla shareholders approve Elon Musk’s record pay package; renewed focus on AI, robotaxi and Optimus
Mild Up effect: Tesla’s emphasis on FSD/robotaxi/Optimus implies continued heavy AI compute needs, a modest sentiment tailwind for NVIDIA given its role in supplying GPUs.
Mild Up effect: {'Defense': ['Lockheed Martin', 'Raytheon', 'Northrop Grumman', 'General Dynamics', 'BAE Systems', 'Thales']}
Mild Up effect: Senate to vote Friday to advance deal to end U.S. government shutdown — Politico
Mild Up effect: Ending/avoiding a shutdown reduces disruption to Pentagon procurement and payments, su